In [20]:
import pandas as pd
import numpy as np

This is the initial load of the data from the csv files, loading the coordinates and the trips. Once both files are loaded, we take care of the multiple spellings of the stations names in the trips file by using the coordinates file, matching by the station id, deleting the extra columns and repeating for for the other set of stations (start and end of journey). Once done we save to a csv file to load from here next time and avoid doing all the steps again. 

In [ ]:
coord = pd.read_csv("./data/coordinatesClean.csv")
trips = pd.read_csv("./data/tripsclean.csv")
trips.rename(columns = {'EndStation.Id':'stationid'}, inplace = True)
temp = pd.merge(trips, coord, on="stationid", how = "left") 
del temp["lat"]
del temp["lon"]
del temp["EndStation.Name"]
temp.rename(columns = {'station':'end_station_name'}, inplace = True)
temp.rename(columns = {"stationid":"end_station_id"}, inplace = True)
temp.rename(columns = {"StartStation.Id":"stationid"}, inplace = True)
temp = pd.merge(temp, coord, on="stationid", how = "left") 
temp.rename(columns = {'station':'start_station_name'}, inplace = True)
temp.rename(columns = {"stationid":"start_station_id"}, inplace = True)
del temp["StartStation.Name"]
del temp["lat"]
del temp["lon"]
temp = temp[["Rental.Id","Bike.Id","Start.Date","End.Date","Duration","start_station_name","start_station_id","end_station_name","end_station_id","Date","Time"]]
temp.to_csv("./data/tripsModified.csv", index = False)

Load the coordinates and the new clean trips file

In [3]:
coord = pd.read_csv(".coordinatesClean.csv")
trips = pd.read_csv(".tripsModified.csv")

In [7]:
del trips["Date"]
trips["Start.Date"] = pd.to_datetime(trips["Start.Date"])
trips["End.Date"] = pd.to_datetime(trips["End.Date"])
# Extract month and year
trips["year"] = trips["Start.Date"].dt.year
trips["month"] = trips["Start.Date"].dt.month
# Extract day of the week
trips["dayofweek"] = trips["Start.Date"].dt.dayofweek
trips["namedayweek"] = trips["Start.Date"].dt.day_name

In [8]:
trips.dtypes

Rental.Id                      int64
Bike.Id                        int64
Start.Date            datetime64[ns]
End.Date              datetime64[ns]
Duration                     float64
start_station_name            object
start_station_id               int64
end_station_name              object
end_station_id                 int64
Time                           int64
year                           int64
month                          int64
dayofweek                      int64
namedayweek                   object
dtype: object

In [21]:
trips_by_hour = trips.groupby("Time").size()
trips_by_hour.to_csv("./data/trips_by_hour.csv")
trips_by_hour = pd.read_csv("./data/trips_by_hour.csv")
trips_by_hour.rename(columns = {"0":"Value"}, inplace = True)


In [23]:
trips_by_weekday = trips.groupby("namedayweek").size()
trips_by_weekday.to_csv("./data/trips_by_weekday.csv")
trips_by_weekday = pd.read_csv("./data/trips_by_weekday.csv")
trips_by_weekday.rename(columns={"0":"Values"}, inplace = True)

In [24]:
trips_by_month = trips.groupby(["month", "year"]).size()
trips_by_month.to_csv("./data/trips_by_month.csv")
trips_by_month = pd.read_csv("./data/trips_by_month.csv")
trips_by_month.rename(columns = {"0":"Values"})


,month,year,Values
0,1,2015,664646
1,1,2016,656303
2,1,2017,680504
3,1,2018,720358
4,1,2019,762991
...,...,...,...
91,12,2018,656357
92,12,2019,619111
93,12,2020,665086
94,12,2021,707975


In [25]:
trips_by_year = trips.groupby(["year"]).size()
trips_by_year.to_csv("./data/trips_by_year.csv")
trips_by_year = pd.read_csv("./data/trips_by_year.csv")
trips_by_year.rename(columns = {"0":"Values"})

,year,Values
0,2015,9307969
1,2016,9982560
2,2017,9077896
3,2018,10441836
4,2019,10309965
5,2020,10287039
6,2021,10821288
7,2022,6221692


In [26]:
max_trip_duration = max(trips.Duration)
avg_trip_duration = trips.Duration.mean()

In [27]:
stations_start = pd.DataFrame(trips.groupby("start_station_name").size())
stations_start.to_csv("./data/stations_start.csv")
stations_start = pd.read_csv("./data/stations_start.csv")
stations_start.rename(columns = {"0":"Trips"}, inplace = True)

In [28]:
stations_end = pd.DataFrame(trips.groupby("end_station_name").size())
stations_end.to_csv("./data/stations_end.csv")
stations_end = pd.read_csv("./data/stations_end.csv")
stations_end.rename(columns = {"0":"Trips"}, inplace = True)

In [29]:
popular_journeys = pd.DataFrame(trips.groupby(["start_station_name", "end_station_name"]).size())
popular_journeys.to_csv("./data/popular_journeys.csv")
popular_journeys = pd.read_csv("./data/popular_journeys.csv")
popular_journeys.rename(columns={"0":"Values"}, inplace = True)

In [30]:
len(popular_journeys.loc[popular_journeys["Values"] == 1])

43031

In [31]:
bikes = pd.DataFrame(trips.groupby("Bike.Id").size())
bikes.to_csv("./data/bikes.csv")
bikes = pd.read_csv("./data/bikes.csv")
bikes.rename(columns={"0":"Values"}, inplace = True)


In [32]:
avg_duration_station = trips.groupby("start_station_name").agg({"Duration": "mean"})
avg_duration_station.to_csv("./data/avg_duration_station.csv")
avg_duration_station = pd.read_csv("./data/avg_duration_station.csv")
avg_duration_station.rename(columns={"0":"Values"}, inplace = True)


In [33]:
avg_duration_by_day_week = trips.groupby("dayofweek").agg({"Duration":"mean"})
avg_duration_by_day_week.to_csv("./data/avg_duration_by_day_week.csv")
avg_duration_by_day_week = pd.read_csv("./data/avg_duration_by_day_week.csv")
avg_duration_by_day_week.rename(columns={"0":"Values"}, inplace = True)


In [35]:
avg_duration_by_day_week_month = trips.groupby(["dayofweek","month"]).agg({"Duration":"mean"})
avg_duration_by_day_week_month.to_csv("./data/avg_duration_by_day_week_month.csv")
avg_duration_by_day_week_month = pd.read_csv("./data/avg_duration_by_day_week_month.csv")
avg_duration_by_day_week_month.rename(columns={"0":"Values"}, inplace = True)

In [36]:
avg_duration_by_month = trips.groupby(["month"]).agg({"Duration":"mean"})
avg_duration_by_month.to_csv("./data/avg_duration_by_month.csv")
avg_duration_by_month = pd.read_csv("./data/avg_duration_by_month.csv")
avg_duration_by_month.rename(columns={"0":"Values"}, inplace = True)

In [37]:
avg_duration_by_year = trips.groupby(["year"]).agg({"Duration":"mean"})
avg_duration_by_year.to_csv("./data/avg_duration_by_year.csv")
avg_duration_by_year = pd.read_csv("./data/avg_duration_by_year.csv")
avg_duration_by_year.rename(columns={"0":"Values"}, inplace = True)